In [ ]:
%pip install matplotlib seaborn numpy pandas nibabel nilearn scikit-learn statsmodels torch joblib networkx python-louvain

import os
import glob
import numpy as np

import pandas as pd
import nibabel as nib
from nilearn.input_data import NiftiLabelsMasker
import statsmodels.formula.api as smf
import torch
import torch.nn as nn
import torch.optim as optim
import re
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
%pip install scipy statsmodels
from scipy.stats import shapiro, levene
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import community.community_louvain as community_louvain

In [ ]:
data_dir = '/Users/hetjivani/NIAR_data/raw_data'
labels_csv = '/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/LatSim/demo.csv'
years = [1, 2, 3]

In [ ]:
from nilearn import datasets

# Fetch the Schaefer 2018 atlas (e.g., 100 parcels, 7 networks)
schaefer = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=1)

# Initialize masker with Schaefer maps
data_masker = NiftiLabelsMasker(labels_img=schaefer.maps, standardize=True)  # human‐readable labels for each parcel

In [ ]:
# Function to compute network metrics
def compute_network_measures(fc_matrix):
    G = nx.from_numpy_array(fc_matrix)

    degree_centrality = np.mean(list(nx.degree_centrality(G).values()))
    global_efficiency = nx.global_efficiency(G)
    clustering_coeff = nx.average_clustering(G, weight='weight')
    partition = community_louvain.best_partition(G, weight='weight')
    modularity = community_louvain.modularity(partition, G, weight='weight')

    return degree_centrality, global_efficiency, clustering_coeff, modularity

In [ ]:
records = []
pattern = re.compile(r"Denoised_([pP]?\d{2}|C\d{2})_rs-(\d+)_MNI.*\.nii(?:\.gz)?$")
for filepath in glob.glob(os.path.join(data_dir, 'Denoised_*_rs-*_MNI.nii*')):
    fname = os.path.basename(filepath)
    m = pattern.match(fname)
    if not m:
        continue

    subj, rs_year = m.groups()
    year = int(rs_year)
    if year not in years:
        continue

    # Load and extract time-series
    img = nib.load(filepath)
    ts = data_masker.fit_transform(img)

    # Compute static FC (Pearson correlation)
    corr = np.corrcoef(ts.T)

    # Set negative correlations to zero for network analysis
    corr_net = np.where(corr < 0, 0, corr)

    # Vectorize upper triangle
    triu_idx = np.triu_indices_from(corr, k=1)
    fc_vec = corr[triu_idx]

    # Compute network metrics
    degree_cent, glob_eff, cluster_coeff, mod = compute_network_measures(corr_net)

    # Dynamic FC variance (example calculation, adjust if sliding windows exist)
    dFC_variance = np.var(corr)

    records.append((subj, year, fc_vec, degree_cent, glob_eff, cluster_coeff, mod, dFC_variance))


In [ ]:
# Unpack results
subjects, yrs, fc_vecs, degrees, efficiencies, clusterings, modularities, dFC_vars = zip(*records)

fc_array = np.vstack(fc_vecs)
df = pd.DataFrame(fc_array, columns=[f'f{i}' for i in range(fc_array.shape[1])])

# Add metadata and covariates
df['subject'] = subjects
df['year'] = yrs
df['degree_centrality'] = degrees
df['global_efficiency'] = efficiencies
df['clustering_coeff'] = clusterings
df['modularity'] = modularities
df['dFC_variance'] = dFC_vars

# Merge in labels
labels_df = pd.read_csv(labels_csv)
labels_df['label_enc'] = LabelEncoder().fit_transform(labels_df['Status'])

df = df.merge(labels_df[['SubId', 'label_enc']], left_on='subject', right_on='SubId')
df.drop(columns=['SubId'], inplace=True)

In [ ]:
print(df.head())
print(df.shape)

In [ ]:
def compute_lme_residuals(df, feature_cols, covariates=None):
    df_out = df.copy()
    for feat in feature_cols:
        if covariates:
            formula = f"{feat} ~ year + " + " + ".join(covariates)
        else:
            formula = f"{feat} ~ year"

        try:
            md = smf.mixedlm(formula, df_out, groups=df_out['subject'], re_formula="~year")
            mdf = md.fit()
            resid_col = f"{feat}_resid"
            df_out[resid_col] = df_out[feat] - mdf.fittedvalues
        except Exception as e:
            print(f"Model failed for feature {feat}: {e}")
            continue
    return df_out

# Define feature columns (FC edges) and covariates (network measures)
feature_cols = [c for c in df.columns if c.startswith('f')]
covariate_cols = ['degree_centrality', 'global_efficiency', 'clustering_coeff', 'modularity', 'dFC_variance']

# Compute residuals while controlling for covariates
df_resid = compute_lme_residuals(df, feature_cols, covariates=covariate_cols)

# Prepare residual feature matrix and labels
resid_cols = [f + '_resid' for f in feature_cols]
X_resid = df_resid[resid_cols].values  # shape: (n_samples, n_edges)
y = df_resid['label_enc'].values


*** LME Test ***

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple, List, Optional
from statsmodels.regression.mixed_linear_model import MixedLMResultsWrapper
from scipy.stats import shapiro, levene


def run_edge_diagnostics(df,
                         feature: str,
                         covariates: Optional[List[str]] = None,
                         alpha: float = 0.05,
                         plot: bool = True):
    """
    Evaluates LME model for a given feature (FC edge) with diagnostics.

    Parameters:
    - df : DataFrame with subject, feature, year, and optional covariates
    - feature : column name of FC edge
    - covariates : list of covariate names (e.g., network metrics)
    - alpha : significance threshold
    - plot : whether to generate diagnostic plots

    Returns:
    - dictionary with test results
    """

    # Construct formula
    fixed_effects = "year"
    if covariates:
        fixed_effects += " + " + " + ".join(covariates)

    f_alt = f"{feature} ~ {fixed_effects}"
    f_null = f"{feature} ~ 1"

    try:
        alt = smf.mixedlm(f_alt, df, groups=df["subject"], re_formula="~year").fit(reml=False, method="lbfgs")
        null = smf.mixedlm(f_null, df, groups=df["subject"], re_formula="1").fit(reml=False, method="lbfgs")
    except Exception as e:
        print(f"{feature}: model fitting error → {e}")
        return None

    if np.isnan(alt.llf) or np.isnan(null.llf):
        print(f"{feature}: model did not converge; skipping")
        return None

    # Likelihood Ratio Test
    lr = 2 * (alt.llf - null.llf)
    dfd = alt.df_modelwc - null.df_modelwc
    p_lr = ss.chi2.sf(lr, dfd)

    # R² Calculation
    var_f = np.var(alt.fittedvalues)
    var_re = np.trace(alt.cov_re)
    var_e = alt.scale
    r2_marg = var_f / (var_f + var_re + var_e)
    r2_cond = (var_f + var_re) / (var_f + var_re + var_e)

    # Residuals
    resid = alt.resid
    fitted = alt.fittedvalues

    # Shapiro-Wilk Test (Normality)
    _, p_shapiro = shapiro(resid)

    # Levene’s Test (Homoscedasticity between groups)
    if 'group' in df.columns:
        g1 = resid[df["group"] == 'control']
        g2 = resid[df["group"] == 'patient']
        _, p_levene = levene(g1, g2)
    else:
        p_levene = np.nan

    # Plots
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(8, 3))
        sm.qqplot(resid, line='s', ax=axes[0])
        axes[0].set_title("QQ plot")
        axes[1].scatter(fitted, resid, s=10)
        axes[1].axhline(0, ls='--')
        axes[1].set_title("Residual vs Fitted")
        plt.tight_layout()
        plt.show()

    # Output
    # print(f"=== Edge {feature} ===")
    # print(f"AIC null  = {null.aic:.2f}")
    # print(f"AIC alt   = {alt.aic:.2f}")
    # print(f"LR χ² = {lr:.2f} (df={dfd})  p = {p_lr:.3g}")
    # print(f"R² marginal = {r2_marg:.3f},  conditional = {r2_cond:.3f}")
    # print(f"Shapiro-Wilk p = {p_shapiro:.4f}, Levene’s p = {p_levene:.4f}")

    return {
        'feature': feature,
        'lr': lr,
        'p_lr': p_lr,
        'r2_marg': r2_marg,
        'r2_cond': r2_cond,
        'p_shapiro': p_shapiro,
        'p_levene': p_levene,
        'converged': alt.converged
    }

In [ ]:
covariate_cols = ['degree_centrality', 'global_efficiency', 'dFC_variance']  # adapt as needed
results = []

for feat in [c for c in df.columns if c.startswith('f')]:
    res = run_edge_diagnostics(df, feature=feat, covariates=covariate_cols, plot=False)
    if res:
        results.append(res)

df_eval = pd.DataFrame(results)
Evaulate = df_eval.to_csv('edge_diagnostics_results.csv', index=False)

In [ ]:
feature = feature_cols[0]
alt_model = smf.mixedlm(f"{feature} ~ year", df, groups=df["subject"], re_formula="~year").fit(reml=False, method="lbfgs")
print(alt_model.cov_re)      # 2×2 matrix

In [ ]:
def fit_edge(df, col, covariates=None):
    """
    Fit a random-intercept LME for edge `col`, return slope β1 and p-value.

    Parameters:
    - df : pd.DataFrame with subject, edge, year, and optionally covariates
    - col : feature/column name to fit (e.g. 'f14')
    - covariates : list of additional fixed-effect covariates (optional)

    Returns:
    - dict with edge, beta (slope for year), and p-value
    """
    try:
        # Construct formula
        fixed_effects = "year"
        if covariates:
            fixed_effects += " + " + " + ".join(covariates)

        formula = f"{col} ~ {fixed_effects}"
        model = smf.mixedlm(formula, df, groups=df["subject"], re_formula="1")
        result = model.fit(reml=False, method="lbfgs")

        beta = result.params["year"]
        pval = result.pvalues["year"]
        return {"edge": col, "beta": beta, "p": pval}

    except Exception as e:
        print(f"Failed to fit model for {col}: {e}")
        return {"edge": col, "beta": np.nan, "p": np.nan}


In [ ]:
covariates = ['degree_centrality', 'global_efficiency', 'dFC_variance']
edge_results = [fit_edge(df, col, covariates=covariates) for col in df.columns if col.startswith('f')]

df_edges = pd.DataFrame(edge_results)


In [ ]:
top5 = stats_df.nsmallest(5, "p").reset_index(drop=True)
print(top5)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(18, 3), sharey=True)

for ax, (_, row) in zip(axes, top5.iterrows()):
    col  = row["edge"]
    beta = row["beta"]
    pval = row["p"]

    # Fit the model for this edge
    model = smf.mixedlm(f"{col} ~ year", df, groups=df["subject"], re_formula="1")
    res = model.fit(reml=False, method="lbfgs")

    # scatter raw data
    ax.scatter(df["year"], df[col], s=10, alpha=0.6)

    # population trend line
    x_line = np.array([df["year"].min(), df["year"].max()])
    y_line = res.params["Intercept"] + beta * x_line
    ax.plot(x_line, y_line, lw=2, color="red")

    ax.set_title(f"{col}\nβ₁={beta:.3g}, p={pval:.2g}")
    ax.set_xlabel("Year")

axes[0].set_ylabel("Edge value (Fisher-z)")
fig.suptitle("Top-5 edges with strongest linear trend", y=1.05, fontsize=14)
plt.tight_layout()


In [ ]:
import statsmodels.formula.api as smf
from scipy.stats import shapiro, chi2

def evaluate_lme_feature(df, feature, covariates=['year']):
    """
    Fit random-intercept + slope LME for a feature with covariates,
    and return diagnostics (Shapiro-Wilk, LR test, AIC, BIC).
    """
    try:
        formula = f"{feature} ~ {' + '.join(covariates)}"

        # Full model: random intercept + slope
        md_full = smf.mixedlm(formula, df, groups=df['subject'], re_formula="~year")
        mdf = md_full.fit(method='lbfgs')

        # Null model: random intercept only
        md_null = smf.mixedlm(formula, df, groups=df['subject'], re_formula="1")
        mdf_null = md_null.fit(method='lbfgs')

        # Manual Likelihood Ratio Test
        lr_stat = 2 * (mdf.llf - mdf_null.llf)
        df_diff = mdf.df_modelwc - mdf_null.df_modelwc
        p_lr = chi2.sf(lr_stat, df_diff)

        # Residual diagnostics
        residuals = mdf.resid
        _, p_shapiro = shapiro(residuals)

        return {
            'feature': feature,
            'shapiro_p': p_shapiro,
            'lr_p': p_lr,
            'AIC_full': mdf.aic,
            'BIC_full': mdf.bic,
            'converged': mdf.converged
        }

    except Exception as e:
        print(f"{feature}: model fitting failed → {e}")
        return {
            'feature': feature,
            'shapiro_p': None,
            'lr_p': None,
            'AIC_full': None,
            'BIC_full': None,
            'converged': False
        }


In [ ]:
covs = ['year', 'degree_centrality', 'global_efficiency', 'dFC_variance']
results = [evaluate_lme_feature(df, f, covariates=covs) for f in df.columns if f.startswith('f')]
df_diagnostics = pd.DataFrame(results)


In [ ]:
# Sort by normality issues
non_normal = df_diagnostics[df_diagnostics['shapiro_p'] < 0.05]
print(f"Number of features violating normality: {len(non_normal)}")

# # Sort by homoscedasticity issues
# heteroscedastic = df_diagnostics[df_diagnostics['levene_p'] < 0.05]
# print(f"Number of features violating homoscedasticity: {len(heteroscedastic)}")

# Check random-effects significance
significant_random_effects = df_diagnostics[df_diagnostics['lr_p'] < 0.05]
print(f"Features benefiting from random slopes: {len(significant_random_effects)}")

# Check convergence issues
convergence_issues = df_diagnostics[~df_diagnostics['converged']]
print(f"Features with convergence issues: {len(convergence_issues)}")

df_diagnostics.head()


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 4))

sns.histplot(df_diagnostics['shapiro_p'], bins=20, ax=axs[0])
axs[0].axvline(0.05, color='red', linestyle='--')
axs[0].set_title('Distribution of Shapiro-Wilk p-values (Normality)')

# sns.histplot(results_df['levene_p'], bins=20, ax=axs[1])
# axs[1].axvline(0.05, color='red', linestyle='--')
# axs[1].set_title('Distribution of Levene’s test p-values (Homoscedasticity)')

sns.histplot(df_diagnostics['lr_p'], bins=20, ax=axs[2])
axs[2].axvline(0.05, color='red', linestyle='--')
axs[2].set_title('Distribution of LR-test p-values (Random Effects)')

plt.tight_layout()
plt.show()


#Sanity Checks

In [ ]:
def fit_and_resid(df, col):
    """Random-intercept-only LME; return residual Series + slope stats."""
    res = smf.mixedlm(f"{col} ~ year", df,
                      groups=df["subject"], re_formula="1"
                     ).fit(reml=False, method="lbfgs")
    series = res.resid.rename(f"{col}_resid")
    return series, res.params["year"], res.pvalues["year"]

# --- parallel loop over all edges -------------------------------------------
out   = Parallel(n_jobs=-1)(
          delayed(fit_and_resid)(df, c) for c in feature_cols
       )

resid_series, betas, pvals = zip(*out)          # unzip tuples
df_resid  = pd.concat([df] + list(resid_series), axis=1, copy=False)

# tidy stats table
stats_df  = pd.DataFrame({
    "edge": feature_cols,
    "beta": betas,
    "p"   : pvals
}).dropna()


In [ ]:
resid_cols = [f"{c}_resid" for c in feature_cols]
sd = df_resid[resid_cols].std()

plt.figure(figsize=(5,3))
sd.hist(bins=50); plt.title("Residual SD per edge"); plt.xlabel("σ"); plt.ylabel("count")


In [ ]:
thr = 0.05
keep_mask   = sd > thr            # tune threshold
keep_edges  = sd[keep_mask].index
print(f"Keeping {len(keep_edges)} / {len(resid_cols)} edges "
      f"({100*keep_mask.mean():.1f}% ) with SD > {thr}")


In [ ]:
X_resid = df_resid[keep_edges].astype("float32").to_numpy()   # (samples × kept_edges)
y       = df_resid["label_enc"].values
print("Residual matrix shape:", X_resid.shape)


In [ ]:
def windowed_fc(ts, win_len=40, step=20):
    """
    ts : ndarray shape (T, N)  -- T time points, N ROIs
    Returns ndarray  (W, N, N) where W is number of windows.
    """
    T, N = ts.shape
    if T < win_len:
        return np.empty((0, N, N), dtype=np.float32)

    windows = []
    for start in range(0, T - win_len + 1, step):
        seg = ts[start:start + win_len]          # win_len × N
        c   = np.corrcoef(seg.T)                 # N × N Pearson-r
        windows.append(np.arctanh(np.clip(c, -0.9999, 0.9999)))  # Fisher-z
    return np.stack(windows, axis=0)             # W × N × N


In [ ]:
sid   = df["subject"].sample(1).iloc[0]
rows  = df_resid["subject"] == sid
ts    = df_resid.loc[rows, list(keep_edges)].to_numpy()
fc_ws = windowed_fc(ts)

print("Window-wise FC std:", fc_ws.std())

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

gkf   = GroupKFold(n_splits=5)
aucs  = []

for train, test in gkf.split(X_resid, y, groups=df_resid["subject"]):
    clf = LogisticRegression(max_iter=500, n_jobs=-1)
    clf.fit(X_resid[train], y[train])
    aucs.append(roc_auc_score(y[test], clf.predict_proba(X_resid[test])[:,1]))

print("Log-Reg ROC-AUC:", np.mean(aucs), "±", np.std(aucs))


In [ ]:
# Option 1: Save as DataFrame with column names
pd.DataFrame(X_resid, columns=keep_edges).to_csv("output.csv", index=False)


In [ ]:
# =============================================================================
#  Dynamic FC extractor  –  from 4-D NIfTI → sliding-window FC tensors
# =============================================================================
# CONFIG (edit if your paths or atlas differ)
DATA_DIR      = "/Users/hetjivani/NIAR_data/raw_data"            # folder with your *.nii / *.nii.gz
OUT_DIR       = "/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/LatSim/Dynamic_Fc"       # where the tensors will live
ATLAS_N_ROI   = 100                          # Schaefer granularity; or 400, 100 …
WIN_LEN       = 40                           # window length in TRs (e.g. 40 × 2 s = 80 s)
STEP          = 20                           # stride in TRs
CLIP_R        = 0.9999                       # avoid arctanh(±1)
N_JOBS        = -1                           # use all CPU cores

# -----------------------------------------------------------------------------
import os, glob, pathlib, numpy as np, pandas as pd
from joblib import Parallel, delayed
from nilearn import datasets, masking
from nilearn.input_data import NiftiLabelsMasker

# 1 ── Grab atlas once
schaefer = datasets.fetch_atlas_schaefer_2018(n_rois=ATLAS_N_ROI, yeo_networks=7)
masker   = NiftiLabelsMasker(labels_img=schaefer.maps,
                             standardize=True, detrend=True, low_pass=0.08, high_pass=0.01,
                             t_r=2.0)                        # ← set t_r to your actual TR in seconds

# 2 ── Helper: sliding-window FC on an ROI time-series matrix
def windowed_fc(ts, win=WIN_LEN, step=STEP, clip=CLIP_R):
    T, N = ts.shape
    if T < win:
        return np.zeros((0, N, N), dtype=np.float32)
    wins = []
    for start in range(0, T - win + 1, step):
        seg = ts[start:start + win]                          # win × N
        c   = np.corrcoef(seg.T)
        wins.append(np.arctanh(np.clip(c, -clip, clip)))     # Fisher-z
    return np.stack(wins).astype("float32")                  # W × N × N

# 3 ── Worker for one file
def process_nifti(nii_path):
    sid   = pathlib.Path(nii_path).stem.split("_")[0]        # customise to your naming
    ts    = masker.fit_transform(nii_path)                   # T × N
    dFC   = windowed_fc(ts)
    if dFC.shape[0] == 0:
        return None
    out_f = pathlib.Path(OUT_DIR) / f"{sid}_dFC.npy"
    np.save(out_f, dFC)
    return {"subject": sid, "T": ts.shape[0], "W": dFC.shape[0], "file": str(out_f)}

# 4 ── Main loop (parallel)
pathlib.Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
nii_files = glob.glob(os.path.join(DATA_DIR, "*.nii*"))
meta_rows = Parallel(n_jobs=N_JOBS, verbose=5)(
    delayed(process_nifti)(p) for p in nii_files
)
meta_df = pd.DataFrame([m for m in meta_rows if m is not None])
meta_df.to_csv(os.path.join(OUT_DIR, "index.csv"), index=False)

print("✓ Finished.  Tensor files in", OUT_DIR)
print(meta_df.head())


In [ ]:
DYNAMIC_FC_DIR = "/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/LatSim/Dynamic_Fc"
MANIFEST_CSV   = os.path.join(DYNAMIC_FC_DIR, "index.csv")

In [ ]:
if os.path.exists(MANIFEST_CSV):
    meta_df = pd.read_csv(MANIFEST_CSV)
else:
    files = sorted(glob.glob(os.path.join(DYNAMIC_FC_DIR, "*_dFC.npy")))
    meta_df = pd.DataFrame({
        "subject": [pathlib.Path(f).stem.split("_")[0] for f in files],
        "file"   : files
    })
    # Optional: count windows T if you like
    meta_df.to_csv(MANIFEST_CSV, index=False)
    print(f"Manifest written → {MANIFEST_CSV}")

print(meta_df.head())

In [ ]:
def vec_upper(mat: np.ndarray) -> np.ndarray:
    """Return upper-triangle (k=1) of a square matrix as 1-D vector."""
    return mat[np.triu_indices_from(mat, k=1)]

sub_stats = []
for _, row in meta_df.iterrows():
    dFC  = np.load(row["file"])          # (W, N, N)
    W, N = dFC.shape[0], dFC.shape[1]
    if W == 0:
        continue
    # stack window vectors  →  (W, n_edges)
    edge_vecs = np.stack([vec_upper(w) for w in dFC])
    edge_sd   = edge_vecs.std(axis=0)    # variability per edge
    sub_stats.append({
        "subject"      : row["subject"],
        "n_windows"    : W,
        "mean_edge_sd" : edge_sd.mean(),
        "max_edge_sd"  : edge_sd.max()
    })

sub_df = pd.DataFrame(sub_stats)
print("\n=== Per-subject dynamic variability ===")
print(sub_df.sort_values("mean_edge_sd", ascending=False).head())
sub_df.to_csv(os.path.join(DYNAMIC_FC_DIR, "subject_variability.csv"), index=False)

In [ ]:
all_vecs = []
for _, row in meta_df.iterrows():
    dFC = np.load(row["file"])
    all_vecs.append(np.stack([vec_upper(w) for w in dFC]))
all_vecs = np.vstack(all_vecs)           # (total_W, n_edges)

edge_sd_global = all_vecs.std(axis=0)
# Map back to ROI indices
N = dFC.shape[1]
tri_i, tri_j   = np.triu_indices(N, k=1)
top_idx        = np.argsort(edge_sd_global)[-10:][::-1]

top10_df = pd.DataFrame({
    "ROI_i"            : tri_i[top_idx],
    "ROI_j"            : tri_j[top_idx],
    "SD_across_windows": edge_sd_global[top_idx]
})
print("\n=== Top-10 most dynamic edges ===")
print(top10_df)
top10_df.to_csv(os.path.join(DYNAMIC_FC_DIR, "top10_edges.csv"), index=False)


In [ ]:
plt.figure(figsize=(6,4))
plt.hist(edge_sd_global, bins=60, edgecolor="k")
plt.xlabel("Edge SD across windows")
plt.ylabel("Count")
plt.title("Distribution of edge-wise dynamic variability")
plt.tight_layout()
fig_path = os.path.join(DYNAMIC_FC_DIR, "edge_variability_hist.png")
plt.savefig(fig_path, dpi=300)
print(f"\nHistogram saved → {fig_path}")
plt.show()

VISULAISE THE TOP 10 DYNAMIC BRAIN REGIONS.

In [ ]:
TOP10_CSV = pathlib.Path("/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/LatSim/Dynamic_Fc/top10_edges.csv")          # produced by the previous script
ATLAS_N_ROI = 100

In [ ]:
from nilearn import plotting

atlas = datasets.fetch_atlas_schaefer_2018(n_rois=ATLAS_N_ROI, yeo_networks=7)
node_coords = plotting.find_parcellation_cut_coords(labels_img=atlas['maps'])

In [ ]:
top10 = pd.read_csv(TOP10_CSV)

# Edge weights (for thickness / colour scale)
edge_weights = top10["SD_across_windows"].values
edge_weights_norm = (edge_weights - edge_weights.min()) / (edge_weights.max() - edge_weights.min())

# Build an empty adjacency matrix and fill the 10 edges
N = ATLAS_N_ROI
adj = np.zeros((N, N))
for (i, row) in top10.iterrows():
    adj[int(row["ROI_i"]), int(row["ROI_j"])] = edge_weights_norm[i]
    adj[int(row["ROI_j"]), int(row["ROI_i"])] = edge_weights_norm[i]  # symmetric

In [ ]:
node_strength = adj.sum(axis=0)
node_strength_norm = (node_strength - node_strength.min()) / (
    node_strength.max() - node_strength.min() + 1e-6
)

# Map node strength → a dark-blue colour map (higher = darker)
from matplotlib.cm import get_cmap
cmap = get_cmap("Blues")
node_colors = [cmap(s) for s in node_strength_norm]

In [ ]:
import numpy as np

node_colors_arr = np.array(node_colors)  # Ensure correct shape for plot_connectome

display = plotting.plot_connectome(
    adj,
    node_coords,
    node_color=node_colors_arr,
    edge_cmap="Reds",
    edge_vmin=0,
    edge_vmax=1,
    edge_threshold="0%",      # draw all 10 edges
    node_size=30 + 120 * node_strength_norm,  # emphasise important nodes
    title="Top-10 Most Dynamic Edges (Schaefer-200)",
)
fig_path = "top10_dynamic_edges.png"
display.savefig(fig_path, dpi=300)
display.close()
print(f"Saved visualisation to {fig_path}")

# DAST GCN

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

# ------------------- Settings and Load Data -------------------
TENSOR_PATH = "/Users/hetjivani/NIAR/Neuro-Imaging-Analysis-and-Research/GroupE/LatSim/Dynamic_Fc/Denoised_dFC.npy"
dFC = np.load(TENSOR_PATH).astype("float32")  # Load dynamic FC tensor with shape (windows, N, N)
print("Shape of loaded dFC tensor:", dFC.shape)
W = dFC.shape[0]

# Define train/test split percentages
YEAR_SPLIT = [0.0, 0.66, 1.0]  # 0-66% for train, 66-100% for test
w1, w2 = int(YEAR_SPLIT[1] * W), int(YEAR_SPLIT[2] * W)

# Split data into train and test tensors
train_tensor = dFC[:w1]                # Training data: first 66%
# For test, include last SEQ_LEN_IN windows from train to allow sliding window input
SEQ_LEN_IN = 5
test_tensor = dFC[w1 - SEQ_LEN_IN : w2]

print("Train tensor shape:", train_tensor.shape)
print("Test tensor shape:", test_tensor.shape)

# ------------------- Normalization (Important to Avoid Data Leakage) -------------------
# We normalize only on training data to avoid leaking info from test set into training

# Reshape 3D tensor to 2D: (windows, features) so scaler can fit
train_reshaped = train_tensor.reshape(train_tensor.shape[0], -1)
scaler = StandardScaler()
scaler.fit(train_reshaped)  # Fit scaler only on training data

# Transform train and test data with the same scaler parameters
train_norm = scaler.transform(train_reshaped).reshape(train_tensor.shape)
test_reshaped = test_tensor.reshape(test_tensor.shape[0], -1)
test_norm = scaler.transform(test_reshaped).reshape(test_tensor.shape)

# Replace original tensors with normalized versions
train_tensor = train_norm
test_tensor = test_norm

# ------------------- Dataset with Sliding Window -------------------
class WindowForecastDS(Dataset):
    def __init__(self, tensor, seq_len):
        self.tensor = tensor
        self.seq_len = seq_len

    def __len__(self):
        # Number of samples is total windows minus sequence length
        return len(self.tensor) - self.seq_len

    def __getitem__(self, idx):
        # Input: seq_len consecutive windows
        x = self.tensor[idx : idx + self.seq_len]
        # Target: the window immediately following the input sequence
        y = self.tensor[idx + self.seq_len]
        return torch.from_numpy(x), torch.from_numpy(y)

# ------------------- DataLoader for Training and Testing -------------------
train_dl = DataLoader(WindowForecastDS(train_tensor, SEQ_LEN_IN),
                      batch_size=32, shuffle=True)   # Shuffle train data for stochasticity

test_dl = DataLoader(WindowForecastDS(test_tensor, SEQ_LEN_IN),
                     batch_size=32, shuffle=False)   # Do NOT shuffle test data to preserve order

print(f"Number of training samples: {len(train_dl.dataset)}")
print(f"Number of testing samples: {len(test_dl.dataset)}")


In [ ]:
class DASTBlock(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.temp = nn.Conv2d(d, d, (3,1), padding=(1,0), groups=d)
        self.ag   = nn.Linear(d, d)
        self.act  = nn.ReLU()
    def forward(self, x):              # (B,T,N,d)
        b,t,n,d = x.shape
        x = x.permute(0,3,1,2)         # B,d,T,N
        x = self.temp(x).permute(0,2,3,1)
        x = self.ag(x)
        return self.act(x)

import torch
import torch.nn as nn

class DASTForecaster(nn.Module):
    def __init__(self, n_roi, d=64, k=2):
        """
        Args:
            n_roi (int): Number of ROIs (regions of interest).
            d (int): Hidden dimension size.
            k (int): Number of DASTBlocks stacked.
        """
        super().__init__()
        self.n_roi = n_roi
        self.inp = nn.Linear(n_roi, d)
        self.blks = nn.ModuleList([DASTBlock(d) for _ in range(k)])
        self.out = nn.Linear(d, n_roi)  # Output predicts one row of the FC matrix

    def forward(self, x):
        """
        Forward pass.

        Args:
            x (torch.Tensor): Input tensor of shape (B, T, N, N), where
                              B=batch size, T=timesteps, N=n_roi.

        Returns:
            torch.Tensor: Predicted FC matrices of shape (B, n_roi, n_roi).
        """
        assert x.dim() == 4, f"Input must be 4D tensor (B,T,N,N), got {x.shape}"
        b, t, n, n2 = x.shape
        assert n == self.n_roi and n2 == self.n_roi, f"Input spatial dims must be n_roi ({self.n_roi}), got ({n}, {n2})"
        x = (x - x.mean()) / x.std()  # standardize globally or per sample

        x = torch.tanh(x)  # Map values back to [-1, 1] range

        # Reshape to (B*T*N, N) for linear input
        x = x.view(b * t, n, n)           # (B*T, N, N)
        x = self.inp(x)                   # (B*T, N, d)
        x = x.view(b, t, n, -1)           # (B, T, N, d)

        # Pass through stacked DASTBlocks
        for blk in self.blks:
            x = blk(x)  # each block expects (B, T, N, d)

        # Temporal mean pooling over T dimension
        x = x.mean(dim=1)  # (B, N, d)

        x = self.out(x)  # (B, N, n_roi)

        # Permute to (B, n_roi, n_roi) for output FC matrix
        x = x.permute(0, 2, 1)  # (B, n_roi, n_roi)

        return x

model = DASTForecaster(N_ROI, d=64, k=2).to(DEVICE)
opt   = torch.optim.Adam(model.parameters(), lr=LR)
mse   = nn.MSELoss()
best  = 1e9


In [ ]:
# Split train_tensor into train and validation sets (e.g. 80% train and 20% validation)
val_ratio = 0.2
val_size = int(len(train_tensor) * val_ratio)
train_size = len(train_tensor) - val_size

train_data = train_tensor[:train_size]
val_data = train_tensor[train_size:]

train_dl = DataLoader(WindowForecastDS(train_data, SEQ_LEN_IN), batch_size=BATCH, shuffle=True)
val_dl = DataLoader(WindowForecastDS(val_data, SEQ_LEN_IN), batch_size=BATCH, shuffle=False)

# EarlyStopping parameters
patience = 5          # Number of epochs we wait before stopping if there is no improvement
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(1, EPOCHS+1):
    model.train()
    running_train = 0
    for xb, yb in tqdm(train_dl, desc=f"Epoch {epoch:02d} - Training", leave=False):
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad()
        loss = mse(model(xb), yb)
        loss.backward()
        opt.step()
        running_train += loss.item() * xb.size(0)
    train_mse = running_train / len(train_dl.dataset) / N_ROI**2
    print(f"Epoch {epoch:02d} | Train MSE/edge = {train_mse:.6f}")

    # Validation phase
    model.eval()
    running_val = 0
    with torch.no_grad():
        for xb, yb in val_dl:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            val_pred = model(xb)
            loss_val = mse(val_pred, yb)
            running_val += loss_val.item() * xb.size(0)
    val_mse = running_val / len(val_dl.dataset) / N_ROI**2
    print(f"Epoch {epoch:02d} | Validation MSE/edge = {val_mse:.6f}")

    # EarlyStopping check
    if val_mse < best_val_loss:
        best_val_loss = val_mse
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_dast_forecaster.pt")
        print("Validation loss improved, model saved.")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation loss for {epochs_no_improve} epochs.")
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break


In [ ]:
# Set model to evaluation mode
model.eval()

# Lists to store predictions, true targets, and RMSEs for each sliding window
preds = []
targets = []
errors = []

with torch.no_grad():
    # Loop over all possible sliding windows in the test data
    for i in range(len(test_tensor) - SEQ_LEN_IN):
        # Extract context sequence of length SEQ_LEN_IN starting from index i
        test_context = torch.from_numpy(test_tensor[i:i+SEQ_LEN_IN][None, ...]).float().to(DEVICE)

        # The ground-truth future adjacency matrix (the target we want to predict)
        true_target = torch.from_numpy(test_tensor[i+SEQ_LEN_IN][None, ...]).float().to(DEVICE)

        # Predict the next adjacency matrix based on the context
        pred = model(test_context)

        # Compute RMSE per edge for this prediction
        err = torch.sqrt(mse(pred, true_target) / N_ROI**2).item()

        # Store predicted and true values (converted back to NumPy)
        preds.append(pred.cpu().numpy()[0])
        targets.append(true_target.cpu().numpy()[0])
        errors.append(err)

# Stack all predicted and true adjacency matrices into arrays
preds = np.stack(preds)
targets = np.stack(targets)

# Save all predictions and corresponding ground-truths for future analysis
np.save("predicted_Y3_all_windows.npy", preds)
np.save("true_Y3_all_windows.npy", targets)

# Print average RMSE per edge across all test windows
print(f"Average RMSE/edge over test windows: {np.mean(errors):.6f}")
